In [2]:
import os
import sys
import glob

## Read the hit table and extract the pdb information (% identity witht the compared sequence, e-value):

In [3]:
with open("./hit_table.txt", "r") as f:
    data = [el.rstrip("\n").split("\t") for el in f.readlines() if el[0] != "#"]
data = [el for el in data if el != ['']]
#data
names = [el[1] for el in data]
per_identitys = [float(el[2]) for el in data]
e_values = [float(el[-3]) for el in data]
#
for i, el in enumerate(names):
    print("system: {} \t identity: {} \t e-value: {}".format(names[i], per_identitys[i], e_values[i]))

hit_results = {el: {"%identity": per_identitys[i], "e-value": e_values[i]} for i, el in enumerate(names)}

## select the best hits by e-value, sequence identity etc
### download the pdbs
### transform the pdbs into fasta
## make the input text for MAFFT (homolog_sequences.fasta)

In [4]:
selected_hits = [(el[0:4], el[5]) if len(el) > 4 else (el[0:4], 'A') for el in hit_results.keys() if (el != "TvLDH" and (hit_results[el]["e-value"] <= 1e-10 and hit_results[el]["%identity"] >= 50.0))]

os.system("rm ./homolog_sequences.fasta")
# for each hit:
for (pdb, chain) in selected_hits:
    print("PDB: {}   chain: {}".format(pdb, chain))
    # download the pdb structure if not present already
    if not glob.glob("../Structures/{}.pdb".format(pdb)):
        print("downloading {}.pdb".format(pdb))
        os.system("curl -s -f https://files.rcsb.org/download/{}.pdb -o ../Structures/{}.pdb".format(pdb, pdb))
        #os.system("gzip -d ./structures/{}.pdb.gz".format(pdb))
    # make fasta sequence
    if not glob.glob("../Sequences/{}.fasta".format(pdb)):
        os.system("./pdb2fasta.py -pdb ../Structures/{}.pdb -out ../Sequences/{}_{}.fasta -chain {}".format(pdb, pdb, chain, chain))

    # cat all the sequences in a single file:
    os.system("cat ../Sequences/{}_{}.fasta >> ./homolog_sequences.fasta".format(pdb, chain))
    os.system("echo "" >> ./homolog_sequences.fasta".format(pdb))